# This notebook helps to create the B tree in form of a table

### First createing a dict with each word across all the con files and taking its count

In [1]:
import pandas as pd
import numpy as np
import glob
import re
from nltk.corpus import stopwords
import string
import operator
import itertools

In [2]:
d = dict()
stop = stopwords.words('english') + list(string.punctuation)

In [3]:
for filename in glob.glob('samples/*.con'):
#     print("****************************************************************")
#     print(filename)
#     print("****************************************************************")
    with open(filename) as f:
        for line in f:
            if not line:
                pass
            else:
                if len(line.split("\"")) == 5:
                    temp = line.split("\"")[1]
                    x = temp.translate(None, string.punctuation)
                    temp = x.strip()
        #             temp_arr = [i for i in temp.split() if i not in stop]
                    if temp in d:
                        d[temp] += 1

                    else:
                        d[temp] = 0
                        d[temp] += 1
            


In [4]:
df = pd.DataFrame.from_dict(d, orient='index')
df.columns=['Values']

In [5]:
df = df.sort_values(by=['Values'], ascending=False)

In [6]:
df.head()

,Values
type num diabetes,321
type num diabetes mellitus,306
treatment,267
the study,245
insulin,233


In [110]:
df.drop(df.head(4).index, inplace=True)

In [9]:
df.head

<bound method DataFrame.head of                                                     Values
type num diabetes                                      321
type num diabetes mellitus                             306
treatment                                              267
the study                                              245
insulin                                                233
metformin                                              224
type 2 diabetes mellitus                               178
diabetes                                               162
type 2 diabetes                                        158
type 1 diabetes                                        145
hba1c 0                                                140
hba1c 5                                                138
type 1 diabetes mellitus                               118
body mass index bmi                                     95
hemoglobin                                              89
medications             

### Start creating the table

In [3]:
lst = list(itertools.product([0, 1], repeat=6))

In [11]:
table = {}
for i in lst:
    table[i] = []

In [12]:
table

{(0, 0, 0, 0, 0, 0): [],
 (0, 0, 0, 0, 0, 1): [],
 (0, 0, 0, 0, 1, 0): [],
 (0, 0, 0, 0, 1, 1): [],
 (0, 0, 0, 1, 0, 0): [],
 (0, 0, 0, 1, 0, 1): [],
 (0, 0, 0, 1, 1, 0): [],
 (0, 0, 0, 1, 1, 1): [],
 (0, 0, 1, 0, 0, 0): [],
 (0, 0, 1, 0, 0, 1): [],
 (0, 0, 1, 0, 1, 0): [],
 (0, 0, 1, 0, 1, 1): [],
 (0, 0, 1, 1, 0, 0): [],
 (0, 0, 1, 1, 0, 1): [],
 (0, 0, 1, 1, 1, 0): [],
 (0, 0, 1, 1, 1, 1): [],
 (0, 1, 0, 0, 0, 0): [],
 (0, 1, 0, 0, 0, 1): [],
 (0, 1, 0, 0, 1, 0): [],
 (0, 1, 0, 0, 1, 1): [],
 (0, 1, 0, 1, 0, 0): [],
 (0, 1, 0, 1, 0, 1): [],
 (0, 1, 0, 1, 1, 0): [],
 (0, 1, 0, 1, 1, 1): [],
 (0, 1, 1, 0, 0, 0): [],
 (0, 1, 1, 0, 0, 1): [],
 (0, 1, 1, 0, 1, 0): [],
 (0, 1, 1, 0, 1, 1): [],
 (0, 1, 1, 1, 0, 0): [],
 (0, 1, 1, 1, 0, 1): [],
 (0, 1, 1, 1, 1, 0): [],
 (0, 1, 1, 1, 1, 1): [],
 (1, 0, 0, 0, 0, 0): [],
 (1, 0, 0, 0, 0, 1): [],
 (1, 0, 0, 0, 1, 0): [],
 (1, 0, 0, 0, 1, 1): [],
 (1, 0, 0, 1, 0, 0): [],
 (1, 0, 0, 1, 0, 1): [],
 (1, 0, 0, 1, 1, 0): [],
 (1, 0, 0, 1, 1, 1): [],


### Now that we have the table, loading the inclusion_exclusion.csv and searching for the top keywords in each file